# Gerador de lat e log 

....

## importação dos dados

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from numpy import dtype

In [ ]:
#importação de dados
df_original = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/DADOS_SISREG.csv', 
    dtype={'CNES_EXECUTANTE':'str','IBGE_USUARIO':'str','CEP_USUARIO':'str', 'CEP_UNID_EXECUTANTE':'str'}) 
df_cep = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
df_cep.columns = ['CEP','UF','CIDADE','BAIRRO','LOGRADOURO']  #http://cep.la/baixar

## identificando CEP's unicos

### CEP's do USUARIO

In [ ]:
#criando um DataFrame somente os CEP's dos usuarios.
df_ceps_usuario = df_original['CEP_USUARIO']
df_ceps_usuario = pd.DataFrame(df_ceps_usuario)
df_ceps_usuario.drop_duplicates(inplace=True)
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)


In [ ]:
#quantidade de CEP's localizado na base 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

### CEP's da UNIDADE EXECUTANTE

In [ ]:
#criando um DataFrame somente os CEP's das unidades.
df_ceps_cnes = df_original['CEP_UNID_EXECUTANTE']
df_ceps_cnes = pd.DataFrame(df_ceps_cnes)
df_ceps_cnes.drop_duplicates(inplace=True)
df_ceps_cnes.dropna(inplace=True)
df_ceps_cnes.head(5)

In [ ]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_cnes.count())

## TRATAMENTO COM OS CEP's DOS USUARIOS

In [ ]:
#localizando endereço do usuario com a base do CEP's BR
df_ceps_usuario = pd.merge(df_ceps_usuario, df_cep, left_on=['CEP_USUARIO'],right_on=['CEP'] , how='left')

In [ ]:
#criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO']+ " - " + df_ceps_usuario['CIDADE']+ " - " + df_ceps_usuario['UF']
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

In [ ]:
#localizando lat e log (PROCESSO DEMORADO)
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head(10)

In [ ]:
#verificar LOCALIZAÇÃO lat e log não localizada (limpeza)
df_ceps_usuario.dropna(inplace=True)

In [ ]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario.head(10)

In [ ]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

## GERAÇÃO DE ARQUIVO CSV - CEP'S com latitude e longitude 

In [ ]:
df_ceps_tratado = df_ceps_usuario.filter(items=['CEP','UF','CIDADE','BAIRRO','LOGRADOURO','LATITUDE_USUARIO','LONGITUDE_USUARIO'])
df_ceps_tratado

In [ ]:
df_ceps_tratado.to_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_LAT_LON.csv')